In [1]:
year = 2023
day = 20

In [131]:
from aocd import submit
from aocd.models import Puzzle
from functools import reduce
import numpy as np

puzzle = Puzzle(year=year, day=day)
data = puzzle.input_data
# data = puzzle.examples[0].input_data

# data = """broadcaster -> a, b, c
# %a -> b
# %b -> c
# %c -> inv
# &inv -> a"""

# data = """broadcaster -> a
# %a -> inv, con
# &inv -> b
# %b -> con
# &con -> output"""



data = data.strip()

data = data.split("\n")

data = {k[1:].strip(): (k[0], [i.strip() for i in v.split(",")]) for k, v in (d.split("->") for d in data)}

data

{'gv': ('%', ['lq', 'pm']),
 'rv': ('%', ['jd', 'nh']),
 'nh': ('%', ['rs', 'jd']),
 'vt': ('&', ['tj']),
 'zv': ('%', ['pm', 'gv']),
 'gh': ('%', ['jd', 'vd']),
 'hh': ('%', ['bf', 'qm']),
 'kx': ('%', ['nf']),
 'st': ('%', ['pm', 'zc']),
 'bh': ('%', ['qm', 'pv']),
 'sk': ('&', ['tj']),
 'hl': ('%', ['nf', 'pn']),
 'mt': ('%', ['st', 'pm']),
 'jd': ('&', ['ts', 'gh', 'vd', 'dc', 'xc']),
 'zm': ('%', ['hm']),
 'pv': ('%', ['vv']),
 'zf': ('%', ['nf', 'cz']),
 'xc': ('&', ['tj']),
 'bf': ('%', ['qm']),
 'ts': ('%', ['sg']),
 'ht': ('%', ['ch', 'nf']),
 'pb': ('%', ['rv', 'jd']),
 'nx': ('%', ['fc']),
 'mb': ('%', ['mt']),
 'mh': ('%', ['jd', 'pb']),
 'lc': ('%', ['bh']),
 'xg': ('%', ['mb', 'pm']),
 'vd': ('%', ['dc']),
 'roadcaster': ('b', ['gh', 'dl', 'xg', 'fb']),
 'sg': ('%', ['mh', 'jd']),
 'qq': ('%', ['ts', 'jd']),
 'dl': ('%', ['nf', 'sv']),
 'vv': ('%', ['sm', 'qm']),
 'zc': ('%', ['tb']),
 'sr': ('%', ['zv', 'pm']),
 'dc': ('%', ['gb']),
 'cz': ('%', ['nf', 'zm']),
 'rs': ('%

In [132]:
from collections import defaultdict
conjunction_mem = defaultdict(lambda: defaultdict(int))
for k, v in data.items():
    for n in v[1]:
        if n == "output":
            continue
        if data[n][0] == "&":
            conjunction_mem[n][k] = 0

conjunction_mem

KeyError: 'rx'

In [133]:
inv_mem = {k: 0 for k, v in data.items() if v[0] == "%"}
inv_mem

{'gv': 0,
 'rv': 0,
 'nh': 0,
 'zv': 0,
 'gh': 0,
 'hh': 0,
 'kx': 0,
 'st': 0,
 'bh': 0,
 'hl': 0,
 'mt': 0,
 'zm': 0,
 'pv': 0,
 'zf': 0,
 'bf': 0,
 'ts': 0,
 'ht': 0,
 'pb': 0,
 'nx': 0,
 'mb': 0,
 'mh': 0,
 'lc': 0,
 'xg': 0,
 'vd': 0,
 'sg': 0,
 'qq': 0,
 'dl': 0,
 'vv': 0,
 'zc': 0,
 'sr': 0,
 'dc': 0,
 'cz': 0,
 'rs': 0,
 'hm': 0,
 'gd': 0,
 'gb': 0,
 'xf': 0,
 'tb': 0,
 'sm': 0,
 'fb': 0,
 'lq': 0,
 'pn': 0,
 'fc': 0,
 'ch': 0,
 'lg': 0,
 'sv': 0,
 'xb': 0,
 'dr': 0}

In [138]:
def reset_inv_mem():
    for k in inv_mem:
        inv_mem[k] = 0
    return inv_mem

def reset_conjunction_mem():
    for k in conjunction_mem:
        for k2 in conjunction_mem[k]:
            conjunction_mem[k][k2] = 0
    return conjunction_mem

In [155]:
# %: flip flop
# b: broadcast
# &: 'and' of all inputs

pulse_list = [("button", "roadcaster", 0)]
MAX_ITER = 10_000

# def simulate(pulse_list, inv_mem, conjunction_mem):

inv_mem = reset_inv_mem()
conjunction_mem = reset_conjunction_mem()
total_pulses_high = 0
total_pulses_low = 0

pulses_per_round = []

for round in range(1, 10_000):

    pulse_list = [("button", "roadcaster", 0)]

    pulses_high = 0
    pulses_low = 1 # initial pulse from button
    print_round = False
    for i in range(MAX_ITER):
        new_pulse_list = []
        for sender, node, val in pulse_list:
            if node not in data:
                continue
            if node == "rx":
                print("rx", val)
            node_type, connections = data[node]
            if node_type == "b":
                new_pulses = [(node, n, 0) for n in connections]
            elif node_type == "%":
                if not val:
                    inv_mem[node] = 1 - inv_mem[node]
                    new_pulses = [(node, n, inv_mem[node]) for n in connections]
                else:
                    new_pulses = []
            elif node_type == "&":
                conjunction_mem[node][sender] = val
                # print(node, conjunction_mem[node], all(conjunction_mem[node].values()))
                output = 0 if all(conjunction_mem[node].values()) else 1
                new_pulses = [(node, n, output) for n in connections]
            else:
                raise ValueError
            new_pulse_list.extend(new_pulses)
        # for p in new_pulse_list:
        #     print(f"{p[0]} -{'high' if p[2] else 'low'}-> {p[1]}")
        pulse_list = new_pulse_list
        pulses_high += sum(p[2] for p in pulse_list)
        pulses_low += sum(1 - p[2] for p in pulse_list)
        if any(conjunction_mem["tj"].values()) and not print_round:
            print(round, i, conjunction_mem["tj"].values())
            print_round = True
        if not pulse_list:
            # print("ran in {} iterations".format(i))
            break
    else:
        raise ValueError("Too many iterations")
    pulses_per_round.append((pulses_high, pulses_low))

    total_pulses_high += pulses_high
    total_pulses_low += pulses_low

total_pulses_high*total_pulses_low

3917 4 dict_values([0, 1, 0, 0])
3931 4 dict_values([0, 0, 0, 1])
3943 4 dict_values([1, 0, 0, 0])
4057 4 dict_values([0, 0, 1, 0])
7834 4 dict_values([0, 1, 0, 0])
7862 4 dict_values([0, 0, 0, 1])
7886 4 dict_values([1, 0, 0, 0])
8114 4 dict_values([0, 0, 1, 0])


82365929720

In [157]:
set(pulses_per_round)

{(32, 11),
 (32, 12),
 (33, 9),
 (37, 16),
 (38, 14),
 (38, 15),
 (39, 14),
 (40, 13),
 (43, 19),
 (43, 20),
 (44, 17),
 (44, 18),
 (44, 19),
 (45, 17),
 (46, 16),
 (46, 18),
 (47, 16),
 (47, 26),
 (47, 27),
 (48, 24),
 (48, 25),
 (49, 23),
 (50, 22),
 (51, 20),
 (51, 21),
 (51, 23),
 (51, 24),
 (52, 19),
 (52, 20),
 (52, 21),
 (53, 19),
 (54, 29),
 (55, 26),
 (55, 27),
 (56, 24),
 (56, 27),
 (57, 23),
 (57, 24),
 (57, 27),
 (57, 28),
 (58, 23),
 (58, 24),
 (58, 25),
 (59, 22),
 (59, 34),
 (60, 31),
 (60, 32),
 (61, 29),
 (61, 30),
 (62, 27),
 (62, 29),
 (62, 30),
 (62, 32),
 (63, 27),
 (63, 29),
 (63, 30),
 (64, 27),
 (64, 32),
 (65, 29),
 (65, 30),
 (66, 27),
 (66, 35),
 (66, 36),
 (67, 33),
 (67, 36),
 (68, 31),
 (68, 33),
 (68, 34),
 (69, 31),
 (69, 34),
 (69, 35),
 (70, 31),
 (70, 34),
 (71, 32),
 (72, 34),
 (72, 39),
 (73, 37),
 (73, 40),
 (74, 36),
 (74, 37),
 (74, 38),
 (74, 39),
 (75, 35),
 (75, 36),
 (75, 37),
 (76, 34),
 (76, 35),
 (76, 36),
 (76, 39),
 (77, 37),
 (78, 36),


In [154]:
# inputs to tj cycle every x rounds
# 3917 4 dict_values([0, 1, 0, 0])
# 3931 4 dict_values([0, 0, 0, 1])
# 3943 4 dict_values([1, 0, 0, 0])
# 4057 4 dict_values([0, 0, 1, 0])
import math
# this round will trigger tj
math.lcm(3917, 3931, 3943, 4057)

246313604784977

In [158]:
3917* 3931* 3943* 4057

246313604784977

In [137]:

total_pulses_high = 0
total_pulses_low = 0
for i in range(1000):
    # if any(conjunction_mem["tj"].values()):
    #     print(i, conjunction_mem["tj"])
    # if i % 100 == 0:
    # print({k: dict(v) for k, v in conjunction_mem.items()})
    # print(dict(conjunction_mem["tj"]))
    # for k in conjunction_mem["tj"]:
    #     print(k, dict(conjunction_mem[k]))
    # print()
    inv_mem, conjunction_mem, pulses_high, pulses_low = simulate(pulse_list, inv_mem, conjunction_mem)
    total_pulses_high += pulses_high
    total_pulses_low += pulses_low

print(total_pulses_high, total_pulses_low)

0 1000


In [115]:
conjunction_mem["tj"]

defaultdict(int, {'vt': 0, 'sk': 0, 'xc': 0, 'kk': 0})

In [105]:
inv_mem = reset_inv_mem()
conjunction_mem = reset_conjunction_mem()

inv_mem, conjunction_mem, pulses_high, pulses_low = simulate(pulse_list, inv_mem, conjunction_mem)

In [108]:
45137*18137

818649769